# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
import logging
import os
import csv


In [2]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset



ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'camels-exp'
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AUCVNXZZL to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: final-prj
Azure region: westus2
Subscription id: 0c66ad45-500d-48af-80d3-0039ebf1975e
Resource group: rgp


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# my code
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

final-prj
rgp
westus2
0c66ad45-500d-48af-80d3-0039ebf1975e


In [4]:
# my code
experiment_name = 'camels-exp'
project_folder = './dmik'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
camels-exp,final-prj,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
# my code
dataset = ws.datasets['camels'] 
df = dataset.to_pandas_dataframe()
df.describe()

,Target,EQTA,EQTL,LLRTA,LLRGL,OEXTA,INCEMP,ROA,ROE,TDTL,TDTA,TATA
count,7020.000000,7020.000000,7020.000000,7020.000000,7020.000000,7020.000000,7014.000000,7020.000000,7020.000000,7020.000000,7020.000000,7020.000000
mean,0.019516,0.107825,8.025950,0.012320,0.021934,0.024020,33.658510,0.002020,-0.234058,44.756417,0.835683,0.176412
std,0.138338,0.048877,573.594468,0.009366,0.160890,0.030903,1156.779875,0.015031,11.397990,3147.677966,0.080119,0.142363
min,0.000000,-0.160659,-0.195857,0.000000,0.000000,-0.012004,-3639.467742,-0.295750,-887.458333,0.000000,0.000000,0.000000
25%,0.000000,0.087487,0.125263,0.007216,0.012119,0.018253,3.084559,0.000907,0.009412,1.126635,0.805493,0.066298
50%,0.000000,0.101018,0.156656,0.010040,0.015915,0.022036,18.162698,0.004832,0.045176,1.273882,0.850135,0.148018
75%,0.000000,0.121013,0.212105,0.014293,0.022124,0.026400,34.348039,0.008417,0.078245,1.527407,0.883593,0.258563
max,1.000000,0.968116,47829.250000,0.161906,12.250000,2.164806,73600.000000,0.173673,21.963100,260238.500000,1.151905,0.868327


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = 'cmp'

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Existing compute target.')

except:
    print('Creating compute target.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

print(compute_target.get_status())

Existing compute target.
{
  "errors": [],
  "creationTime": "2021-03-12T16:36:50.492879+00:00",
  "createdBy": {
    "userObjectId": "49e75006-b9ac-415c-9176-f83c59d4bf26",
    "userTenantId": "d689239e-c492-40c6-b391-2c5951d31d14",
    "userName": "Mikhaylov, Dmitry"
  },
  "modifiedTime": "2021-03-12T16:39:52.341446+00:00",
  "state": "Running",
  "vmSize": "STANDARD_DS3_V2"
}


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'norm_macro_recall'
}

# TODO: Put your automl config here


automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Target",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
# Submit your automl run
from azureml.widgets import RunDetails
run = experiment.submit(config=automl_config, show_output=True)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

Running on remote.
Running on remote compute: cmp
Parent Run ID: AutoML_490852a9-29a8-4669-9062-51eb24f6b59c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
# Retrieve and save your best automl model.
best_run, fitted_model = run.get_output()
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-core, training version:1.23.0, current version:1.22.0
Package:azureml-dataprep, training version:2.10.1, current version:2.9.1
Package:azureml-dataprep-native, training version:30.0.0, current version:29.0.0
Package:azureml-dataprep-rslex, training version:1.8.0, current version:1.7.0
Package:azureml-dataset-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-defaults, training version:1.23.0, current version:1.22.0
Package:azureml-interpret, training version:1.23.0, current version:1.22.0
Package:azureml-mlflow, training version:1.23.0, current version:1.22.0
Package:azureml-pipeline-core, training version:1.23.0, current version:1.22.0
Package:azureml-telemetry, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-client, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-runtime, training version:1.23.0, current versio

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_split=None,
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.29105263157894734,
     

In [21]:
print(best_run)

Run(Experiment: camels-exp,
Id: AutoML_490852a9-29a8-4669-9062-51eb24f6b59c_39,
Type: azureml.scriptrun,
Status: Completed)


In [22]:
#TODO: Save the best model
# Register the model produced by AutoML
import joblib

joblib.dump(value=fitted_model, filename="fitted_automl_model.joblib")
automl_model = best_run.register_model(model_name='automl_model.pkl', model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Delete the cluster instance
AmlCompute.delete(compute_target)